# **FIAP Pós-Tech - IA para Devs**

# **Tech Challenge - Fase 2**

## Grupo 14 (1IADT) - Alunos

* Marcelo Henriques da Fonseca - RM353865
* Marcos Lopes da Silva Junior - RM353763
* Ricardo Báfica Pontes - RM353866

# **Descrição do projeto**

## O problema

- O desafio consiste em projetar, implementar e testar um sistema que utilize Algoritmos Genéticos para otimizar uma função ou resolver um problema complexo de otimização. Você pode escolher problemas como otimização de rotas, alocação de recursos e design de redes neurais.

## Requisitos do projeto

1. **Definição do Problema:** escolha um problema real que possa ser resolvido por meio de otimização genética. Descreva o problema, os objetivos e os critérios.
2. **Testes e resultados:** realize testes para demonstrar a eficácia do algoritmo. Compare os resultados obtidos com métodos de solução convencionais.
3. **Documentação:** forneça uma documentação completa do projeto, incluindo descrições do problema, datalhes da implementação do algorítmo, análises de resultados e conclusões.  

# **Introdução**

## Descrição do problema

Uma banda de música está planejando uma turnê por algumas das capitais do Brasil. O grupo quer visitar 11 capitais de diversos estados e retornar ao ponto de partida ao final da turnê.

## Abordagem

A fim de minimizar a distância percorrida e os custos da viagem, utilizaremos tanto um algorítmo genético quanto uma solução de força bruta para determinar uma melhor rota.

Ao final, faremos uma comparação entre a eficiência dos dois modelos em resolver o problema.

## Os objetivos e os critérios de sucesso

1.   Escolher uma rota que minimize a distância total percorrida.
2.   Não visitar nenhuma capital mais de uma vez.
3.   Retornar ao ponto inicial ao final da turnê.



## Detalhes do Algoritmo Genético

**Indivíduo** = conjunto de capitais (uma rota)

**Fitness** = distância total da rota (quanto menor, melhor)

**Objetivo** = encontrar uma rota que minimize a distância total a ser percorrida

# Dados iniciais

In [32]:
# Importanto bibliotecas
import math
import random
import itertools
import pandas as pd
import numpy as np
import copy
import pygame
import time

No bloco abaixo, definimos uma lista de capitais que serão visitadas pela banda.

In [33]:
# Definindo as capitais e suas coordenadas (latitude, longitude)

capitais = {
    # 'Rio Branco': (-9.974, -67.8243),
    # 'Maceió': (-9.6498, -35.7089),
    # 'Macapá': (0.0349, -51.0694),
    'Manaus': (-3.1190, -60.0217),
    'Salvador': (-12.9714, -38.5014),
    'Fortaleza': (-3.7172, -38.5434),
    'Brasília': (-15.7801, -47.9292),
    # 'Vitória': (-20.3155, -40.3128),
    'Goiânia': (-16.6869, -49.2643),
    # 'São Luís': (-2.5307, -44.3068),
    # 'Cuiabá': (-15.6010, -56.0974),
    # 'Campo Grande': (-20.4486, -54.6295),
    'Belo Horizonte': (-19.9173, -43.9346),
    # 'Belém': (-1.4558, -48.4902),
    # 'João Pessoa': (-7.1150, -34.8641),
    'Curitiba': (-25.4284, -49.2733),
    # 'Recife': (-8.0476, -34.8770),
    # 'Teresina': (-5.0919, -42.8034),
    'Rio de Janeiro': (-22.9068, -43.1729),
    # 'Natal': (-5.7945, -35.2110),
    'Porto Alegre': (-30.0346, -51.2177),
    # 'Porto Velho': (-8.7611, -63.9004),
    # 'Boa Vista': (2.8235, -60.6758),
    # 'Florianópolis': (-27.5954, -48.5480),
    'São Paulo': (-23.5505, -46.6333),
    # 'Aracaju': (-10.9472, -37.0731),
     'Palmas': (-10.1853, -48.3277)
}

# Convertendo para uma tupla de localizações
localizacoes_cidades = tuple(capitais.values())

# Visualização da tupla
print(localizacoes_cidades)

((-3.119, -60.0217), (-12.9714, -38.5014), (-3.7172, -38.5434), (-15.7801, -47.9292), (-16.6869, -49.2643), (-19.9173, -43.9346), (-25.4284, -49.2733), (-22.9068, -43.1729), (-30.0346, -51.2177), (-23.5505, -46.6333), (-10.1853, -48.3277))


# Funções

Função que calcula a distância entre uma cidade1 e uma cidade2 levando em conta a curvatura do planeta Terra.

In [34]:
# função para calcular a distância entre duas cidades dadas suas coordenadas (latitude e longitude)
def calcula_distancia(cidade1, cidade2):
    # extrai as coordenadas das cidades, x = latitude e y = logitude
    x1, y1 = cidade1
    x2, y2 = cidade2

    # Fórmula de Haversine
    # a = sin²(Δφ/2) + cos φ1 ⋅ cos φ2 ⋅ sin²(Δλ/2)
    # c = 2 ⋅ atan2( √a, √(1−a) )
    # d = R ⋅ c
    # onde φ é a latitude, λ a longitude, R é o raio da Terra
    # R = 6371 km
    R = 6371
    a = math.sin((y2 - y1) / 2) ** 2 + math.cos(y1) * math.cos(y2) * math.sin((x2 - x1) / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R * c

    return d

Função para gerar uma população inicial aleatória para a utilização no algoritmo genético.

In [35]:
# população inicial aleatória
def gera_populacao_aleatoria(tamanho_populacao: int):

    populacao = []
    for _ in range(tamanho_populacao):
        populacao.append(random.sample(localizacoes_cidades, len(localizacoes_cidades)))

    return populacao

Função para gerar uma população inicial utilizando KNN (K-Nearest Neighbors) para a utilização no algoritmo genético.

Como a população inicial segue um metódo que busca minimizar distâncias ao invés de ser aleatória, ela acaba sendo mais otimizada e ajuda nosso algoritmo genético a chegar em resultados melhores mais rapidamente.

Não está sendo utilizada no momento, mas permanece no notebook com propósito didático e para testes.

In [36]:
# população inicial usando KNN
def gera_populacao_knn(tamanho_populacao: int):

    populacao = []
    for _ in range(tamanho_populacao):
        cidade_inicial = random.choice(localizacoes_cidades)
        solucao = [cidade_inicial]
        cidades_restantes = set(localizacoes_cidades) - {cidade_inicial}
        while cidades_restantes:
            ultima_cidade = solucao[-1]
            proxima_cidade = min(cidades_restantes, key=lambda x: calcula_distancia(ultima_cidade, x))
            solucao.append(proxima_cidade)
            cidades_restantes.remove(proxima_cidade)
        populacao.append(solucao)

    return populacao

Função que calcula a adaptação de um indivíduo de uma população, retornando a distância total de sua rota.

Isso nos dá um bom parâmetro de comparação para que o algoritmo possa achar as melhores soluções.

Neste caso, os indivíduos melhores adaptados são aqueles que possuem uma distância total menor.

In [37]:
# adaptação do indivíduo
def calcula_adaptacao(individuo):

    distancia_total = 0

    n = len(individuo)
    for i in range(n):
        distancia_total += calcula_distancia(individuo[i], individuo[(i + 1) % n])

    return distancia_total

Função de cruzamento que combina duas soluções (ascendentes) para gerar duas novas soluções (descendentes).

O retorno de dois descendentes aumenta a diversidade genética na população, o que ajuda o algoritmo a explorar um maior número de soluções possíveis.
Ele também faz com que o algoritmo evolua mais rapidamente em direção a melhores soluções e ajuda a manter o tamanho da população constante.

In [38]:
# cruzamento de indivíduos
def cruzamento(ascendente1, ascendente2):
    tam = len(ascendente1)

    # Escolha aleatória dos pontos de corte
    ini, fim = sorted(random.sample(range(tam), 2))

    # Criação de listas vazias para os descendentes
    descendente1 = [None] * tam
    descendente2 = [None] * tam

    # Copia o segmento entre os pontos de corte dos ascendentes para os descendentes
    descendente1[ini:tam+1] = ascendente1[ini:tam+1]
    descendente2[ini:tam+1] = ascendente2[ini:tam+1]

    # Preenche o restante das cidades nos descendentes
    posicoes_faltantes1 = [i for i in range(tam) if not (ini <= i <= fim)]
    posicoes_faltantes2 = [i for i in range(tam) if not (ini <= i <= fim)]

    posicao_atual1, posicao_atual2 = 0, 0

    for cidade in ascendente2:
        if cidade not in descendente1:
            while descendente1[posicoes_faltantes1[posicao_atual1]] is not None:
                posicao_atual1 += 1
            descendente1[posicoes_faltantes1[posicao_atual1]] = cidade
            posicao_atual1 += 1

    for cidade in ascendente1:
        if cidade not in descendente2:
            while descendente2[posicoes_faltantes2[posicao_atual2]] is not None:
                posicao_atual2 += 1
            descendente2[posicoes_faltantes2[posicao_atual2]] = cidade
            posicao_atual2 += 1

    return descendente1, descendente2

A seleção de ascendentes por probabilidade busca que indivíduos mais adaptados sejam selecionados como ascendentes.

Essa seleção é feita de forma bastante rápida, mas tem um fator de aleatoriedade que pode levar o algoritmo a demorar mais a chegar em um resultado ótimo.

Não está sendo utilizada no momento, mas permanece no notebook com propósito didático e para testes.

In [39]:
def seleciona_ascendentes_por_probabilidade(populacao, adaptacao_populacao, num_ascendentes=2):
    # indivíduos com melhor adaptação (menor distância total) têm maior probabilidade de reprodução
    probabilidade = 1 / np.array(adaptacao_populacao)

    # seleção aleatória de ascendentes
    ascendente1, ascendente2 = random.choices(populacao, weights=probabilidade, k=num_ascendentes)

    return ascendente1, ascendente2

A seleção de ascendentes por ranking garante que indivíduos mais adaptados sejam selecionados como ascendentes, permitindo que o algoritmo converja mais rapidamente para soluções melhores.
Essa seleção acaba sendo um pouco mais demorada que a aleatório, visto que há uma ordenação. Porém, em casos de problemas maiores, esse ligeiro aumento do tempo de seleção não é tão significativo, visto que nos ajuda a chegar em resultados melhores.

A seleção por probabilidade também aumenta as chances de indivíduos melhores adaptados serem selecionados nessa metade superior.

O bloco abaixo recebe uma população ordenada pelos indivíduos mais adaptados (menor distância) e seleciona (com uma probabilidade maior para indivíduos com melhor adaptação) indíviduos da metade superior para serem ascendentes. Isso garante uma maior diversidade genética ao invés de escolhermos estritamente os que estão no topo da lista.

Esse método combina os benefícios da seleção por ranking com a seleção por probabilidade, ainda permitindo a possibilidade de seleção de indivíduos menos adaptados para manter a diversidade genética. Esse método é conhecido como Seleção por Torneio com Probabilidade de Seleção.

In [40]:
# Selecionar os ascendentes por ranking
def seleciona_ascendentes_por_ranking(populacao_ordenada, adaptacao_populacao, num_ascendentes=2):
    # Seleciona a metade superior dos indivíduos
    metade_superior_populacao = populacao_ordenada[:len(populacao_ordenada)//2]
    metade_superior_adaptacao = adaptacao_populacao[:len(adaptacao_populacao)//2]

    # Calcular as probabilidades inversas das adaptações
    probabilidade = 1 / np.array(metade_superior_adaptacao)

    # Normalizar as probabilidades para somarem 1
    probabilidade /= np.sum(probabilidade)

    # Seleciona os ascendentes com base no ranking
    ascendentes = random.choices(metade_superior_populacao, weights=probabilidade, k=num_ascendentes)

    return ascendentes

Função de mutação que ajuda a introduzir variação em uma população, evitando que ela fique presa em ótimos locais e fazendo com que o algoritmo explore novas soluções. Duas abordagens foram avaliadas:

1. mutação aleatória: o indivíduo é mutado com a troca aleatória de duas cidades.

2. mutação por embaralhamento: envolve selecionar uma subsequência e embaralhá-la aleatoriamente. Isso mantém a ordem relativa dos elementos fora da subsequência, mas muda a ordem dentro da subsequência.

Diversidade: A mutação por troca aleatória de duas cidades é bastante rápido e tende a manter a diversidade das soluções, pois pode explorar diferentes partes do espaço de busca.

Eficiência: A mutação por embaralhamento pode ser mais lenta em alguns casos, especialmente se a subsequência a ser embaralhada for grande, mas tende a oferecer melhores mudanças na solução.

Optamos por aplicar ambas as mutações, uma para cada um dos dois individuos gerados no cruzamnento.

In [41]:
# mutação aleatória de um indivíduo
def mutacao_aleatoria(individuo_original, probabilidade_mutacao):
    individuo_mutado = copy.deepcopy(individuo_original)
    if random.random() > probabilidade_mutacao:
        return individuo_mutado

    # escolha aleatória das cidades
    posicao1, posicao2 = sorted(random.sample(range(len(individuo_original)), 2))

    # troca as posições
    individuo_mutado[posicao1], individuo_mutado[posicao2] = individuo_mutado[posicao2], individuo_mutado[posicao1]

    return individuo_mutado

In [42]:
# mutação por embaralhamento de um indivíduo
def mutacao_embaralhamento(individuo_original, probabilidade_mutacao):
    individuo_mutado = copy.deepcopy(individuo_original)
    if random.random() > probabilidade_mutacao:
        return individuo_mutado

    # Seleciona dois índices aleatórios
    posicao1, posicao2 = sorted(random.sample(range(len(individuo_original)), 2))

    # Embaralha a subsequência entre os índices
    subseq = individuo_mutado[posicao1:posicao2+1]
    random.shuffle(subseq)
    individuo_mutado[posicao1:posicao2+1] = subseq

    return individuo_mutado


Função que ordena uma população pela adaptação de cada um de seus indivíduos de forma crescente, ou seja, os indivíduos que apresentam menor distância total são os primeiros do vetor.

In [43]:
# ordena a população por melhor adaptação
def ordena_populacao(populacao, adaptacao):
    listas_combinadas = list(zip(populacao, adaptacao))
    listas_combinadas_ordenadas = sorted(listas_combinadas, key=lambda x: x[1])
    populacao_ordenada_por_adaptacao, adaptacao_ordenada = zip(*listas_combinadas_ordenadas)

    return populacao_ordenada_por_adaptacao, adaptacao_ordenada

# Algoritmo Genético

Definição de alguns parâmetros importantes para a execução do algoritmo genético.

"TAMANHO_POPULACAO" define o número de indíviduos de uma população em cada geração. Um tamanho de população maior leva a uma diversidade genética maior e uma exploração de mais possíveis soluções, mas aumenta o custo computacional.

"MAXIMO_DE_GERACOES" define um limite para o número de gerações que o algoritmo genético vai executar antes de parar, evitando execuções excessivamente longas.

"PROBABILIDADE_DE_MUTACAO" define a probabilidade um indivíduo sofrer mutação. Uma probabilidade alta pode aumentar a diversidade genética, mas pode causar muita aleatoriedade. Uma probabilidade baixa, por outro lado, pode fazer com que o algoritmo fique estagnado em uma solução subótima.

"NUMERO_GERACOES_SEM_MELHORA_MAXIMO" define um limite para quantas gerações o algoritmo continuará executando sem encontrar soluções melhores. Ajuda a diminuir o tempo de execução e evitar que o algoritmo continue executando sem progresso significativo. Pode diminuir a capacidade do algoritmo de encontrar boas soluções se for um número muito baixo.

In [44]:
# parâmetros de controle
TAMANHO_POPULACAO = 200
MAXIMO_DE_GERACOES = 1000

PROBABILIDADE_DE_MUTACAO = 0.6

NUMERO_GERACOES_SEM_MELHORA_MAXIMO = 200

# geração para população inicial
populacao = gera_populacao_aleatoria(TAMANHO_POPULACAO)
#populacao = gera_populacao_knn(TAMANHO_POPULACAO)

Execução do algoritmo genético, juntando todos os passos tomados até agora em busca de uma solução pra o problema.

In [45]:
contador_geracoes = itertools.count(start=1)

melhor_solucao = None
melhor_adaptacao = float('inf')

ultima_melhor_adaptacao = float('inf')
numero_geracoes_sem_melhora = 0

# loop principal
ini_gem = time.time()
while True:

    geracao = next(contador_geracoes)

    adaptacao_populacao = [calcula_adaptacao(individuo) for individuo in populacao]

    populacao, adaptacao_populacao = ordena_populacao(populacao, adaptacao_populacao)

    melhor_adaptacao = adaptacao_populacao[0]
    melhor_solucao = populacao[0]

    # condição de parada
    if melhor_adaptacao == ultima_melhor_adaptacao:
        numero_geracoes_sem_melhora += 1
    else:
        numero_geracoes_sem_melhora = 0
    ultima_melhor_adaptacao = melhor_adaptacao

    # imprime a melhor adaptação
    print(f"Geração: {geracao:03d} | adaptação: {melhor_adaptacao}")

    # condição de saída
    if geracao > MAXIMO_DE_GERACOES or numero_geracoes_sem_melhora >= NUMERO_GERACOES_SEM_MELHORA_MAXIMO:
        break

    # elitismo, o melhor individuo será mantido na próxima população
    nova_populacao = [populacao[0]]

    while len(nova_populacao) < TAMANHO_POPULACAO:
        # seleção de ascendentes aleatória
        #ascendente1, ascendente2 = seleciona_ascendentes_por_probabilidade(populacao, adaptacao_populacao)

        # seleção de ascendentes por ranking
        ascendente1, ascendente2 = seleciona_ascendentes_por_ranking(populacao, adaptacao_populacao)

        # nascimento de dois novos indivíduos
        descendente1, descendente2 = cruzamento(ascendente1, ascendente2)

        # eventual mutação do indivíduo 1
        descendente1 = mutacao_aleatoria(descendente1, PROBABILIDADE_DE_MUTACAO)
        # eventual mutação do indivíduo 2
        descendente2 = mutacao_embaralhamento(descendente2, PROBABILIDADE_DE_MUTACAO)

        # incorpora os novos indivíduos à nova população
        nova_populacao.append(descendente1)
        nova_populacao.append(descendente2)

    populacao = nova_populacao

fim_gem = time.time()

Geração: 001 | adaptação: 72722.84796596372
Geração: 002 | adaptação: 68639.72819587219
Geração: 003 | adaptação: 68639.72819587219
Geração: 004 | adaptação: 68639.72819587219
Geração: 005 | adaptação: 68639.72819587219
Geração: 006 | adaptação: 62511.0932000106
Geração: 007 | adaptação: 62511.0932000106
Geração: 008 | adaptação: 62511.0932000106
Geração: 009 | adaptação: 62511.0932000106
Geração: 010 | adaptação: 59227.316348876586
Geração: 011 | adaptação: 59227.316348876586
Geração: 012 | adaptação: 59227.316348876586
Geração: 013 | adaptação: 56417.47953703359
Geração: 014 | adaptação: 56417.47953703359
Geração: 015 | adaptação: 53133.70268589958
Geração: 016 | adaptação: 53133.70268589958
Geração: 017 | adaptação: 53133.70268589958
Geração: 018 | adaptação: 53133.70268589958
Geração: 019 | adaptação: 53133.70268589958
Geração: 020 | adaptação: 53133.70268589958
Geração: 021 | adaptação: 53133.70268589958
Geração: 022 | adaptação: 53133.70268589958
Geração: 023 | adaptação: 53133.7

# **Resultado do algorítmo genético**

Faz uma busca reversa na lista de capitais para nos mostrar a rota encontrada e a distância total.

In [46]:
# Mostrar a melhor solução
nomes_capitais = list(capitais.keys())
rota_ag = [nomes_capitais[localizacoes_cidades.index(cidade)] for cidade in melhor_solucao]
print("Melhor Rota:", " -> ".join(rota_ag))
distancia_total_alg_gem = melhor_adaptacao
print("Distância Total: ", distancia_total_alg_gem, "km")
print(f"Tempo de execução: {fim_gem - ini_gem} segundos")

Melhor Rota: Goiânia -> Rio de Janeiro -> Fortaleza -> Porto Alegre -> Salvador -> São Paulo -> Belo Horizonte -> Manaus -> Brasília -> Curitiba -> Palmas
Distância Total:  53133.70268589958 km
Tempo de execução: 4.089007616043091 segundos


# **Resolvendo o problema com força bruta**

Resolução do problema com "força bruta", testando diversas rotas até encontrar a melhor solução para o problema.

In [47]:
# função para calcular a distância total de um percurso
def calcular_distancia_total(percurso, cidades):
    distancia_total = 0
    for i in range(len(percurso) - 1):
        distancia_total += calcula_distancia(cidades[percurso[i]], cidades[percurso[i + 1]])
    # adiciona a distância de volta ao ponto de partida
    distancia_total += calcula_distancia(cidades[percurso[-1]], cidades[percurso[0]])
    return distancia_total

In [48]:
# função para encontrar o percurso de menor distância usando força bruta
def menor_percurso(cidades):
    indices_cidades = list(range(len(cidades)))
    melhor_percurso = None
    menor_distancia = float('inf')

    # gerar todas as permutações possíveis dos índices das cidades
    cont = 0
    for percurso in itertools.permutations(indices_cidades):
        cont += 1
        # calcular a distância total do percurso
        distancia_atual = calcular_distancia_total(percurso, cidades)
        if distancia_atual < menor_distancia:
            menor_distancia = distancia_atual
            melhor_percurso = percurso
    print(cont)

    return melhor_percurso, menor_distancia

In [49]:
v_capitais = list(capitais.values())
ini_fb = time.time()
melhor_percurso, menor_distancia = menor_percurso(v_capitais)
fim_fb = time.time()

39916800


Faz uma busca reversa na lista de capitais para nos mostrar a rota encontrada e a distância total.

In [50]:
# Mostrar o menor percurso
rota_fb = [nomes_capitais[indice_cidade] for indice_cidade in melhor_percurso]
print("Melhor Rota:", " -> ".join(rota_fb))
print("Menor distância: ", menor_distancia, "km")
print(f"Tempo de execução: {fim_fb - ini_fb} segundos")

Melhor Rota: Manaus -> Belo Horizonte -> São Paulo -> Salvador -> Porto Alegre -> Fortaleza -> Rio de Janeiro -> Goiânia -> Palmas -> Curitiba -> Brasília
Menor distância:  53133.702685899574 km
Tempo de execução: 738.9720985889435 segundos


# **Conclusão**

## **Comparação: Algorítmo Genético vs Força Bruta**

In [59]:
# tempo do algoritmo genético
print(f"Tempo de execução do algoritmo genético: {fim_gem - ini_gem} segundos")

# rota do algoritmo genético
print("Melhor Rota do Algoritmo Genético:", " -> ".join(rota_ag))

print(" ----------------------------------------------------------------------------------------------------------------------------------------------------------------------- ")

# tempo da força bruta
print(f"Tempo de execução da força bruta: {fim_fb - ini_fb} segundos")

# rota da força bruta
print("Melhor Rota da Força Bruta:", " -> ".join(rota_fb))

print(" ----------------------------------------------------------------------------------------------------------------------------------------------------------------------- ")

# diferença em quilometros do algoritmo genético para o de força bruta
acuracia_alg_gem = menor_distancia - distancia_total_alg_gem
print("Diferença: ", abs(acuracia_alg_gem), "km")

Tempo de execução do algoritmo genético: 4.089007616043091 segundos
Melhor Rota do Algoritmo Genético: Goiânia -> Rio de Janeiro -> Fortaleza -> Porto Alegre -> Salvador -> São Paulo -> Belo Horizonte -> Manaus -> Brasília -> Curitiba -> Palmas
 ----------------------------------------------------------------------------------------------------------------------------------------------------------------------- 
Tempo de execução da força bruta: 738.9720985889435 segundos
Melhor Rota da Força Bruta: Goiânia -> Rio de Janeiro -> Fortaleza -> Porto Alegre -> Salvador -> São Paulo -> Belo Horizonte -> Manaus -> Brasília -> Curitiba -> Palmas
 ----------------------------------------------------------------------------------------------------------------------------------------------------------------------- 
Diferença:  7.275957614183426e-12 km


Podemos notar que existe uma diferença muito grande entre o tempo de execução do algoritmo genético e o tempo de execução da "força bruta".

Olhando a diferença de distância entre as soluções (rotas) encontradas pelos dois métodos, podemos verificar que ela não existe, já que o algoritmo genético encontra a melhor rota neste conjunto de cidades. É importante destacar que nem sempre ele irá achar a melhor solução.

O algoritmo genético é escrito e otimizado com o propósito de encontrar uma boa solução para o nosso problema da turnê em um tempo reduzido.
Ele preza pela eficiência, nos permitindo encontrar uma solução otimizada em um tempo significativamente menor.

O método de "força bruta" vai encontrar a melhor solução para o problema, mas para isso ele deve testar TODAS as soluções possíveis, o que pode demorar muito tempo e ter um grande custo computacional.
Este tempo ainda é aumentado exponencialmente ao adicionarmos mais e mais cidades ao problema, o que pode tornar esse método extremamente custoso ou, em muitos casos, inviável.

# **Considerações finais**


Neste experimento, o algoritmo genético é quase 100 vezes mais rápido que o método de força bruta.

Ainda que o algoritmo genético não garanta ao usuário a solução exata para um problema em todos os casos, este experimento demonstra o seu poder de resolução diante de situações em que o método de força bruta possua elevado consumo de processamento computacional e tempo longo para as necessidades do cliente/usuário.

O "case" explorado nesse notebook aborda um problema de tamanho pequeno, mas a busca de uma solução por algoritmo genético pode ser aplicada em muitos problemas grandes do dia-a-dia.

Esse exemplo pode ser adaptado para planejar rotas de entrega de produtos, viagens de negócios, rotas de coleta de lixo, roteamente de ambulâncias, roteamente de equipes de inspeção, e várias outras aplicações em diversos setores onde a otimização de rotas é necessária.